In [1]:
#import necessary libraries
import cv2
import csv 
import mediapipe as mp
import os
from tensorflow.keras.preprocessing.image import img_to_array
# mediapipe settings
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
# field names initialization
fields = [['right_wrist'] ,['right_elbow'], ['right_shoulder'], ['left_wrist'], ['left_elbow'], ['left_shoulder'], ['right_hip'], ['right_knee'], ['right_ankle'], ['left_hip'], ['left_knee'], ['left_ankle']]
right_wrist_x = right_elbow_x = right_shoulder_x = left_wrist_x = left_elbow_x = left_shoulder_x = right_hip_x = right_knee_x = right_ankle_x = left_hip_x = left_knee_x = left_ankle_x = -1
right_wrist_y = right_elbow_y =  right_shoulder_y =  left_wrist_y =  left_elbow_y =  left_shoulder_y = right_hip_y =  right_knee_y = right_ankle_y = left_hip_y = left_knee_y = left_ankle_y = -1

# get the video, set camera id for live camera use case, for example set 0 if you have one camera, i.e. web cam
cap = cv2.VideoCapture(r'E:\MRH\Video\Pocket 3rd Part.mp4')
List = [] # for storing target class
img_num = 1 #for gathering window_len amount image frame
window_len = 32 # set window length as per your traininng
with open(r'E:\\MRH\\X_Test.csv','a', newline='') as f1: #open a file, use w as 2nd argument if the file isn't created yet. Use a when you want to append multiple videos' data
    with mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:
        writer=csv.writer(f1, delimiter=',')
        #writer.writerow(fields)
        while cap.isOpened():
            success, image = cap.read()
            if image is None:
                break
            if not success:
                print("Ignoring empty camera frame.")
                # If loading a video, use 'break' instead of 'continue'.,
                continue

            # To improve performance, optionally mark the image as not writeable to,
            # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image)

            image_new = image.copy()
            image_height, image_width, _ = image_new.shape
            if not results.pose_landmarks:
                continue
            right_wrist_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].x * image_width
            right_wrist_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].y * image_height
            
            right_elbow_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW].x * image_width
            right_elbow_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW].y * image_height

            right_shoulder_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * image_width
            right_shoulder_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * image_height

            left_wrist_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].x * image_width
            left_wrist_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].y * image_height

            left_elbow_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW].x * image_width
            left_elbow_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW].y * image_height

            left_shoulder_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * image_width
            left_shoulder_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * image_height

            right_hip_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP].x * image_width
            right_hip_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP].y * image_height

            right_knee_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE].x * image_width
            right_knee_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE].y * image_height

            right_ankle_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].x * image_width
            right_ankle_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].y * image_height

            left_hip_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].x * image_width
            left_hip_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].y * image_height

            left_knee_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].x * image_width
            left_knee_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].y * image_height

            left_ankle_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].x * image_width
            left_ankle_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].y * image_height
            
            #change the serial of data points as you seem necessary
            row = [
                [right_wrist_x], [right_wrist_y],
                [right_elbow_x], [right_elbow_y],
                [right_shoulder_x], [right_shoulder_y],
                [left_wrist_x], [left_wrist_y],
                [left_elbow_x], [left_elbow_y],
                [left_shoulder_x], [left_shoulder_y],
                [right_hip_x], [right_hip_y],
                [right_knee_x], [right_knee_y],
                [right_ankle_x], [right_ankle_y],
                [left_hip_x], [left_hip_y],
                [left_knee_x], [left_knee_y],
                [left_ankle_x], [left_ankle_y]
            ]
            if img_num % window_len == 0:
                List.append(1)
            writer.writerow(row)
            img_num = img_num + 1
            
            # Draw the pose annotation on the image.,
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
            # Flip the image horizontally for a selfie-view display.,
            cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
            if cv2.waitKey(5) & 0xFF == 27:
                break
cap.release()
cv2.destroyAllWindows()

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\\\MRH\\\\X_Test.csv'

In [38]:
# this saves the target class data into a csv file
with open(r'E:\\MRH\\Y_Test.csv','a', newline='') as f2:
    writer=csv.writer(f2, delimiter=',')
    for item in List:
        writer.writerow([item])

In [ ]:
"""
            print(
                f'Hand coordinates 16: ('
                f'{results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].x * image_width}, '
                f'{results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].y * image_height})'
            )
            print(
                f'Hand coordinates 14: ('
                f'{results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW].x * image_width}, '
                f'{results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW].y * image_height})'
            )
            print(
                f'Hand coordinates 12: ('
                f'{results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * image_width}, '
                f'{results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * image_height})'
            )
            print(
                f'Hand coordinates 15: ('
                f'{results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].x * image_width}, '
                f'{results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].y * image_height})'
            )
            print(
                f'Hand coordinates 13: ('
                f'{results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW].x * image_width}, '
                f'{results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW].y * image_height})'
            )
            print(
                f'Hand coordinates 11: ('
                f'{results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * image_width}, '
                f'{results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * image_height})'
            )"""